In [3]:
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy import stats

# Задание 1

Оцените эксперимент «Sending email (correct link)» с использованием CUPED. В качестве ковариаты используйте выручку пользователей за 4 недели до эксперимента.

Данные эксперимента «Sending email (correct link)»: 2022-05-03/2022-05-03T12_df_sales.csv и 2022-05-03/experiment_users.csv. Эксперимент проводился с 2022-04-25 по 2022-05-02. Метрика — средняя выручка с клиента.

В качестве ответа введите p-value с точность до 4-го знака после точки.

In [18]:
# загрузка данных
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-05-03/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales = read_database('2022-05-03T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

df_exp_users = read_database('experiment_users.csv')
df_check = pd.read_csv('df_metrics_1000.csv')

In [19]:
df_exp = (
    df_sales[(df_sales['date'] >= datetime(2022, 4, 25)) & (df_sales['date'] <= datetime(2022, 5, 2))]
).copy()

df_user_metrics = df_exp.groupby(['user_id'], as_index=False)['price'].sum().rename(columns={'price': 'metric'})

In [20]:
df_cuped = (
    df_sales[(df_sales['date'] >= datetime(2022, 3, 28)) & (df_sales['date'] < datetime(2022, 4, 25))]
)

df_user_metrics_before = df_cuped.groupby(['user_id'], as_index=False)['price'].sum().rename(columns={'price': 'metric_before'})

In [25]:
df = df_exp_users.merge(df_user_metrics, how='left', on='user_id').fillna(0)
df = df.merge(df_user_metrics_before,  how='left', on='user_id').fillna(0)

In [28]:
df_control = df[df['pilot'] == 0]
df_test = df[df['pilot'] == 1]

In [29]:
def calculate_theta(y_control, y_pilot, x_control, x_pilot):
    """Вычисляем Theta по данным двух групп.

    y_control - значения метрики во время пилота на контрольной группе
    y_pilot - значения метрики во время пилота на пилотной группе
    x_control - значения ковариант на контрольной группе
    x_pilot - значения ковариант на пилотной группе
    """
    y = np.hstack([y_control, y_pilot])
    x = np.hstack([x_control, x_pilot])
    covariance = np.cov(x, y)[0, 1]
    variance = x.var()
    theta = covariance / variance
    return theta

In [30]:
def check_cuped_test(df_control, df_pilot, covariate_column):
    """Проверяет гипотезу о равенстве средних с использованием CUPED.

    covariate_column - название стобца с ковариантой

    return - pvalue.
    """
    theta = calculate_theta(
        df_control['metric'], df_pilot['metric'],
        df_control[covariate_column], df_pilot[covariate_column]
    )
    metric_cuped_control = df_control['metric'] - theta * df_control[covariate_column]
    metric_cuped_pilot = df_pilot['metric'] - theta * df_pilot[covariate_column]
    _, pvalue = stats.ttest_ind(metric_cuped_control, metric_cuped_pilot)
    return pvalue

In [31]:
pvalue = check_cuped_test(df_control, df_test, 'metric_before')
print(f'pvalue с CUPED: {pvalue:0.4f}')

pvalue с CUPED: 0.0539
